In [11]:
import numpy as np
import cv2
import os
import skimage.io as io
from skimage.color import rgb2gray
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt

## Loading Datasets

### Our Dataset

In [16]:
dirname = os.path.abspath('')
datasets_path = os.path.join(dirname, 'Datasets')
people_path = os.path.join(datasets_path, 'Our faces')

def load_our_dataset():
    #  Loop over every directory in people path
    y = dict()
    images = []
    i=0
    min_width = float('inf')
    min_height = float('inf')
    
    for directory in os.listdir(people_path):
        #  Loop over every image in the directory
        for image in os.listdir(os.path.join(people_path, directory)):
            #  Load the image
            img = io.imread(os.path.join(people_path, directory, image))
            img = rgb2gray(img)
            # resize image to width*height
            if (np.max(img) > 1):
                img = img / 255.0
            
            if(img.shape[0] < min_height):
                min_height = img.shape[0]
            if(img.shape[1] < min_width):
                min_width = img.shape[1]
                
            images.append(img)
            y[i] = [directory]
            i+=1
    
    images = np.array([cv2.resize(image, (min_height, min_width))for image in images])
        
    return images,y

### Olivetti Dataset

## Convert images to gray scale and same shape of trained images

In [ ]:
#  ---------------------------- CONVERT IMAGE TO GRAY SCALE AND SAME SHAPE OF TRAINED IMAGES ---------------------------- #

def convert_image(image, width, height):
    # plt.imshow(image, cmap='gray', label='Input image')
    # plt.show()
    # if image is RGB, convert to grayscale
    if len(image.shape) > 2:
        image = image[:,:,:3]
        image = rgb2gray(image)
    # resize image to width*height
    image = cv2.resize(image, (width, height))
    if (np.max(image) > 1):
        image = image / 255.0
    image = image.reshape((width*height, 1))
    # Show image
    # plt.imshow(image.reshape(height, width), cmap='gray', label='Input image')
    # plt.show()

    return image

## Flatten Images

In [ ]:
def flatten_images(images, m, height, width, verbose=False):
    # Flatten images array
    # Each column is an image

    # N^2 * M
    flattened_images = images.reshape(m, -1).T

    if(verbose):
        print(
            f'Shape of training images after flattening: {flattened_images.shape}')

    return flattened_images

## Extract Info from Dataser

In [ ]:
def extract_info_from_our_dataset(images,y,verbose=False):
    m = images.shape[0]
    if(verbose):
        print('Images shape:', images.shape)

    total_images, height, width = images.shape
    if(verbose):
        print(f'Each image has size: {height} x {width}')
        print(70 * '-')

    n_features = height*width
    if(verbose):
        print(f'N^2 = n_features = h x w = {n_features}')
        print(70 * '-')

    if(verbose):
        print('y has shape:', y.shape)

    return images, m, height, width, total_images, n_features, y

## Load Parameters

In [ ]:
def load_parameters():
    omegas = np.load(omegas_path)
    eigenfaces = np.load(eigenfaces_path)
    average_image = np.load(average_image_path)
    return omegas, eigenfaces, average_image

## Predict an Image

In [ ]:
def predict(omegas, test_image, average_image, eigenfaces, height, width, threshold=7):
    # Predict the class of a test image
    # Calculate omega for test image
    omega = np.matmul((test_image - average_image).T, eigenfaces)

    # Calculate distance between omega and all omegas
    distances = np.linalg.norm(omegas - omega, axis=1)

    # Get the index of the minimum distance
    index = np.argmin(distances)

    # Get min distance
    min_distance = distances[index]
    if(min_distance < threshold):
        return index
    else:
        return -1  # Unknown

## Main

In [ ]:
def recognizer_main(input_image):
    # dataset = load_olivetti_dataset()
    # images, _, height, width, _, _, _, _ = extract_info_from_olivetti_dataset(dataset)
    images, y = load_our_dataset()
    images, _, height, width, _, _, _ = extract_info_from_our_dataset(images,y)
    
    omegas, eigenfaces, average_image = load_parameters()
    input_image = convert_image(input_image, width, height)
    plt.imshow(input_image.reshape(height,width), cmap='gray', label='Input image')
    plt.show()

    predicted_index = predict(omegas, input_image, average_image, eigenfaces, height, width, 120)
    if (predicted_index == -1):
        print('Unknown face')
    else:
        print('Known face')
        # show predicted image
        print("Image Index: ", predicted_index)
        print("Target Index: ", y[predicted_index])
        plt.imshow(images[predicted_index],cmap='gray', label='Predicted image')
        plt.show()